<div class="elfjS" data-track-load="description_content"><p>A <strong>subsequence</strong> of a string is a new string that is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters. (i.e., <code>"ace"</code> is a subsequence of <code>"<u>a</u>b<u>c</u>d<u>e</u>"</code> while <code>"aec"</code> is not).</p>

<p>Given two strings <code>source</code> and <code>target</code>, return <em>the minimum number of <strong>subsequences</strong> of </em><code>source</code><em> such that their concatenation equals </em><code>target</code>. If the task is impossible, return <code>-1</code>.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre><strong>Input:</strong> source = "abc", target = "abcbc"
<strong>Output:</strong> 2
<strong>Explanation:</strong> The target "abcbc" can be formed by "abc" and "bc", which are subsequences of source "abc".
</pre>

<p><strong class="example">Example 2:</strong></p>

<pre><strong>Input:</strong> source = "abc", target = "acdbc"
<strong>Output:</strong> -1
<strong>Explanation:</strong> The target string cannot be constructed from the subsequences of source string due to the character "d" in target string.
</pre>

<p><strong class="example">Example 3:</strong></p>

<pre><strong>Input:</strong> source = "xyz", target = "xzyxz"
<strong>Output:</strong> 3
<strong>Explanation:</strong> The target string can be constructed as follows "xz" + "y" + "xz".
</pre>

<p>&nbsp;</p>
<p><strong>Constraints:</strong></p>

<ul>
	<li><code>1 &lt;= source.length, target.length &lt;= 1000</code></li>
	<li><code>source</code> and <code>target</code> consist of lowercase English letters.</li>
</ul>
</div>

In [19]:
from typing import *

In [20]:
class Solution:
    def shortestWay(self, source: str, target: str, verbose: bool = False) -> int:
        """
        Using a trie data structure, this implementation solves the problem
        in O(S^2 + T) time complexity where S is the length of the source
        string and T is the length of the target string.
        Building the trie using O(2^S) memory complexity.
        :param source:  Source word to use to build the target word.
        :param target:  Target word to build using subsequences of the source word.
        :param verbose: If true, print additional information.
        :return:        The number of subsequences of the source word to use to
                        build the target word.
        """
        s = len(source)

        # We can make 2^S words out of the source. We reduce the runtime to O(S^2)
        # by only taking source[i:] and creating a trie for it. Then, source[i-1:]
        # can create its trie by simply using the tries of all the
        # subproblems in front of it.
        trie_subproblems = [{}] * s
        for i in range(s-1, -1, -1):
            # Iterate through the source backwards as each character's trie
            # depends only on the character's ahead of it.

            # ts will hold the solution to the current trie subproblem
            ts = {}

            for j in range(i+1, s):
                # the trie of source[i] is simply a composition of the tries
                # of the subsequences in front of it

                subseq_char = source[j]
                if subseq_char not in ts:
                    # If a character repeats, we only need the character whose
                    # position is closest to index i. Otherwise, we introduce
                    # redundant subtrees.
                    ts[subseq_char] = trie_subproblems[j]

            # insert the solution into the array
            trie_subproblems[i] = ts

        # using the above subproblems, create the root of our trie
        trie = {}
        for ts, c in zip(trie_subproblems, source):
            if c not in trie:
                # Similar to above, if a character repeats, we only
                # need the first instance's trie. Otherwise, we may
                # introduce redundant subtrees or accidentally remove
                # valid subtrees.
                trie[c] = ts

        if verbose:
            print(f"Trie: \n{trie}")

        # we have not started parsing, start prefix as the empty set
        current_prefix = {}
        # number of subsequences of the source word we need to
        # concatenate to recover the target word
        num_subsequences = 0
        for c in target:

            # Get the node of the next character in the prefix. If
            # it does not exist, return None by default
            next_prefix = current_prefix.get(c, None)

            if next_prefix is None and c not in trie:
                # This case is infeasible as c cannot extend
                # our current prefix and c does not exist as the
                # first character of a word in our trie.
                return -1

            if next_prefix is None and c in trie:
                # c cannot extend our current prefix, but it does exist
                # in the trie. Start a new prefix of length 1. Since this
                # is a new prefix, increment the number of subsequences
                # of the source word that we are using.
                num_subsequences += 1
                current_prefix = trie[c]
            else:
                # c can extend our current prefix. Keep fleshing out our
                # current subsequence.
                current_prefix = next_prefix

        return num_subsequences

def main():
    test_cases = {
        "1": {
            "source": "abc",
            "target": "abcbc",
            "expected": 2,
        },
        "2": {
            "source": "abc",
            "target": "acdbc",
            "expected": -1,
        },
        "3": {
            "source": "xyz",
            "target": "xzyxz",
            "expected": 3,
        },
        "4": {
            "source": "aaaaa",
            "target": "aaaaaaaaaaaaa",
            "expected": 3,
        },
        "5": {
            "source": "lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze",
            "target": "lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze",
            "expected": 5,
        }
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        # verbose must be false for test case 5 since its trie is too large
        ret = solution.shortestWay(**targs, verbose=False)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")

main()

test case 1: {'source': 'abc', 'target': 'abcbc'}
Returned: 2, Expected: 2
Passed:True

test case 2: {'source': 'abc', 'target': 'acdbc'}
Returned: -1, Expected: -1
Passed:True

test case 3: {'source': 'xyz', 'target': 'xzyxz'}
Returned: 3, Expected: 3
Passed:True

test case 4: {'source': 'aaaaa', 'target': 'aaaaaaaaaaaaa'}
Returned: 3, Expected: 3
Passed:True

test case 5: {'source': 'lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze', 'target': 'lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze'}
Returned: 5, Expected: 5
Passed:True



This approach sacrifies some memory for time complexity. The time is $O(S + T)$ while the auxiliary space is $O(S)$.

In [21]:
class Solution:
    def shortestWay(self, source: str, target: str, verbose: bool = False) -> int:
        """
        Using a trie data structure, this implementation solves the problem
        in O(S^2 + T) time complexity where S is the length of the source
        string and T is the length of the target string.
        Building the trie using O(2^S) memory complexity.
        :param source:  Source word to use to build the target word.
        :param target:  Target word to build using subsequences of the source word.
        :param verbose: If true, print additional information.
        :return:        The number of subsequences of the source word to use to
                        build the target word.
        """
        s = len(source)
        t = len(target)

        source_set = set([c for c in source])
        for c in target:
            if c not in source_set:
                # if a character exists in the target set that does not appear in the
                # source set, a solution is infeasible
                return -1

        # Has shape (s, 26). Each entry (i, j) marks the first occurrence of the j'th character
        # in the alphabet in the substring source[i:].
        lookup_table = [[-1]*26 for _ in range(s + 1)] # shape (s, 26)
        source_int = [ord(c) - ord('a') for c in source]
        target_int = [ord(c) - ord('a') for c in target]
        for i, j in zip(range(s-1, -1, -1), source_int[::-1]):
            # traverse source backwards to fill in the table

            # copy the entries from the subsequent row
            lookup_table[i][:] = lookup_table[i+1][:]
            # update the first occurrence of the current character
            lookup_table[i][j] = i

        source_iterator = 0
        number_of_subsequences = 1 # first use of our source word
        for j in target_int:
            first_idx = lookup_table[source_iterator][j]
            if first_idx > -1:
                source_iterator = first_idx + 1
            else:
                source_iterator = lookup_table[0][j] + 1
                number_of_subsequences += 1

        return number_of_subsequences

def main():
    test_cases = {
        "1": {
            "source": "abc",
            "target": "abcbc",
            "expected": 2,
        },
        "2": {
            "source": "abc",
            "target": "acdbc",
            "expected": -1,
        },
        "3": {
            "source": "xyz",
            "target": "xzyxz",
            "expected": 3,
        },
        "4": {
            "source": "aaaaa",
            "target": "aaaaaaaaaaaaa",
            "expected": 3,
        },
        "5": {
            "source": "lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze",
            "target": "lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze",
            "expected": 5,
        }
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        # verbose must be false for test case 5 since its trie is too large
        ret = solution.shortestWay(**targs, verbose=False)
        if expected is not None:
            passed = ret == expected
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}\n")

main()

test case 1: {'source': 'abc', 'target': 'abcbc'}
Returned: 2, Expected: 2
Passed:True

test case 2: {'source': 'abc', 'target': 'acdbc'}
Returned: -1, Expected: -1
Passed:True

test case 3: {'source': 'xyz', 'target': 'xzyxz'}
Returned: 3, Expected: 3
Passed:True

test case 4: {'source': 'aaaaa', 'target': 'aaaaaaaaaaaaa'}
Returned: 3, Expected: 3
Passed:True

test case 5: {'source': 'lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze', 'target': 'lkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlzelkynjisqcrfkkhrrwwgdnzziupubqchukdinntwiesphyryfsgkjtngabsnzwlze'}
Returned: 5, Expected: 5
Passed:True

